In [ ]:
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
use role sysadmin;
create database finservam_migration;
grant all on database finservam_migration to role finservam_admin;
grant all on all schemas in database finservam_migration to role finservam_admin;

Create an Azure stage in the UI

In [ ]:
use role finservam_admin;
ls @finservamd/trade;

In [ ]:
create or replace file format my_parquet_format
  type = 'parquet'; -- in this case parquet

In [ ]:
create or replace table finservam_migration.public.trade
  using template (
    select array_agg(object_construct(*))
      from table(
        infer_schema(
          location=>'@finservamd/trades/trades_0_0_0.snappy.parquet',
          file_format=>'my_parquet_format'
        )
      ));

In [ ]:
describe table trade;

In [ ]:
select distinct $1:SYMBOL
from @finservamd/trades/trades_0_0_0.snappy.parquet
(FILE_FORMAT=>'my_parquet_format')
limit 10;

In [ ]:
use warehouse finservam_devops_wh;
alter warehouse finservam_devops_wh set warehouse_size = '2x-large' wait_for_completion = TRUE   enable_query_acceleration = true;

copy into trade
  from @finservamd/trades/
  file_format = (type=parquet)
  match_by_column_name = case_insensitive;

  --size down immediately save credits
alter warehouse finservam_devops_wh suspend;
alter warehouse finservam_devops_wh set warehouse_size = 'xsmall';

In [ ]:
select * from trade limit 10;